In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
#from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

Using TensorFlow backend.
C:\Users\getto\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\getto\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\getto\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\getto\AppData\Roa

In [2]:
dataset = pd.read_csv("data_final.csv")
dataset.dropna(inplace=True)
dataset.head()


,tweet,label
1,RT @colonelkickhead: Another bloody instant re...,none
3,@azzamalirhabi @JihadiA8 This video of the Pes...,none
5,Oh really? No more instant restaurants? THAT'S...,none
7,RT @Benfrancisallen: It hasn't been a good few...,none
9,RT @NoToFeminism: I donâ€™t need femisnsn beca...,none


In [3]:
dataset.shape

(16090, 2)

In [4]:
def preprocess_text(sen):
    sentence = remove_tags(sen)
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [5]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [6]:
X = []
sentences = list(dataset['tweet'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [7]:
#y = dataset['label']

#encoder = LabelBinarizer()
#y = encoder.fit_transform(y)

In [8]:
dataset["label"].unique()

array(['none', 'racism', 'sexism'], dtype=object)

In [9]:
dataset.label.value_counts()

none      11036
sexism     3117
racism     1937
Name: label, dtype: int64

In [10]:
le=LabelEncoder()

In [11]:
le.fit(dataset["label"])

LabelEncoder()

In [12]:
print("\nDataset Labels:(y values)\n",list(le.classes_))


Dataset Labels:(y values)
 ['none', 'racism', 'sexism']


In [13]:
y=le.transform(dataset["label"])
y

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

In [17]:
tfidf_vectorizer.fit(X)

TfidfVectorizer(max_df=0.7, stop_words='english')

In [18]:
tfidf_train = tfidf_vectorizer.transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)

In [19]:
file=open("tokenizer.pickle","wb")
joblib.dump(tfidf_vectorizer,file)
file.close()

In [21]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

In [22]:
print('Original dataset shape %s' % Counter(y_train))

Original dataset shape Counter({0: 8836, 2: 2503, 1: 1533})


In [23]:
ros = RandomOverSampler(random_state=42)
x_res, y_res = ros.fit_resample(tfidf_train, y_train)

In [24]:
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({2: 8836, 0: 8836, 1: 8836})


In [25]:
from lightgbm import LGBMClassifier

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [26]:
classifier_lgbm = LGBMClassifier()
classifier_lgbm.fit(x_res, y_res)

LGBMClassifier()

In [31]:
y_pred_lgbm=classifier_lgbm.predict(x_res)
y_pred_lgbm

array([2, 0, 0, ..., 2, 2, 2])

In [33]:
print("accuracy_score_lgbm: \n",accuracy_score(y_res,y_pred_lgbm)*100)

accuracy_score_lgbm: 
 92.27403048136411


In [34]:
print("confusion_matrix: \n",confusion_matrix(y_res,y_pred_lgbm))

confusion_matrix: 
 [[7927  455  454]
 [ 127 8709    0]
 [1012    0 7824]]


In [35]:
print("classification_report: \n",classification_report(y_res,y_pred_lgbm))

classification_report: 
               precision    recall  f1-score   support

           0       0.87      0.90      0.89      8836
           1       0.95      0.99      0.97      8836
           2       0.95      0.89      0.91      8836

    accuracy                           0.92     26508
   macro avg       0.92      0.92      0.92     26508
weighted avg       0.92      0.92      0.92     26508



In [37]:
file=open("labels.pkl","wb")
joblib.dump(le,file)
file.close()


In [36]:
file=open("model.pkl","wb")
joblib.dump(classifier_lgbm,file)
file.close()

In [85]:
file=open("tokenizer.pkl","wb")
joblib.dump(tokenizer,file)
file.close()